In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [2]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [3]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [5]:
# Create the generator
netG = Generator(ngpu)

netG.apply(weights_init)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [6]:
fitness_map = {netG : 1500}

In [13]:
{key.ngpu: value for key, value in fitness_map.items()}

{1: 1500}

In [ ]:
{key.random_tag: value for key, value in host.gen_error_map.items()}

In [35]:
new_params = {}
mu = 0
sigma = 0.5

In [36]:
for name, param in netG.named_parameters():
    new_params[name] = param.clone()
    if param.dim() == 4:
        random_tensor = torch.from_numpy(np.random.lognormal(mu, sigma, param.size()))
        new_params[name] *= random_tensor
        param.data.copy_(new_params[name])

In [13]:
new_params = {}
mu = 0
sigma = 0.5
random_tensor = np.random.lognormal(mu, sigma)

In [14]:
for name, param in netG.named_parameters():
    new_params[name] = param.clone()
    if param.dim() == 4:
        new_params[name] *= random_tensor
        param.data.copy_(new_params[name])

In [31]:
for name, param in netG.named_parameters():
    print(name, param.size())

main.0.weight torch.Size([100, 512, 4, 4])
main.1.weight torch.Size([512])
main.1.bias torch.Size([512])
main.3.weight torch.Size([512, 256, 4, 4])
main.4.weight torch.Size([256])
main.4.bias torch.Size([256])
main.6.weight torch.Size([256, 128, 4, 4])
main.7.weight torch.Size([128])
main.7.bias torch.Size([128])
main.9.weight torch.Size([128, 64, 4, 4])
main.10.weight torch.Size([64])
main.10.bias torch.Size([64])
main.12.weight torch.Size([64, 3, 4, 4])


In [13]:
possibilities = ['kill_two_thirds', 'kill_one_third', 'kill_no_one']
weights = [0.01, 0.09, 0.9]
choice = random.choices(possibilities, weights)
choice

['kill_no_one']

In [6]:
proportions = [0.1, 0.075, 0.05, 0.025] #possible proportions of the number of weight layers changing
proportion = random.choice(proportions) #randomly choose one, from 1/10th (0.1) to 1/100th (0.01)

In [9]:
weights_layers = []

#with torch.no_grad():
    
#extract the weight layers (no bias layers)
for name, _ in netG.named_parameters():
    if name.find("weight") != -1 :
        weights_layers.append(name)

#randomly choose which weight layers to change 
#Given the network structure, containing always 9 weight layers, and the proportions chosen manually above, 
#we will always end up changing either one weight layer (when proportion= 0.1 or 0.075) or nothing 
#(when proportion= 0.05 or 0.025) thanks to the np.around() below.
#So we have randomness on whether to change or not, if yes which layer (they have different sizes), and multiplying by 0.5 or 2
nb_weight_layers = len(weights_layers)
number_of_weight_layers_to_change = int(np.around(nb_weight_layers*proportion))
weight_layers_to_change = random.sample(weights_layers, number_of_weight_layers_to_change)


new_params = {}


for name, params in netG.named_parameters():
    new_params[name] = params.clone() #clone the parameters to be able to change later (or add torch.no_grad() ?)
    if name in weight_layers_to_change:
        new_params[name] = params * random.choice([0.5, 2])
        params.data.copy_(new_params[name])
    